<h1><center>Laboratorio 3: Clasificación y Clustering 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Nicolás Acevedo
- Nombre de alumno 2: Branco Paineman
- Nombre de alumno 3: Pablo Ubilla

<i>Nota: Nuestro grupo tiene permiso para ser de 3</i>


### **Link de repositorio de GitHub:** `http://....`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Manejo de DataFrames con la libreria `pandas`.
- Exploración del dataset y los resultados a través de visualizaciones interactivas usando `plotly`.
- Clasificación en `scikit-learn`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 4/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar y aprovechar las ventajas que nos ofrece la libreria `pandas`.
- Utilizar `plotly` para obtener información gráfica del dataset.
- Aplicar las ventajas que nos ofrece crear un pipeline.
- Visualizar el funcionamiento de clasificadores.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [75]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
# !pip install umap-learn

# 1. Predicciones futboleras.

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

Para este laboratorio se deberá trabajar con los csv ```FullData.csv```, donde deberán aplicar algoritmos de Clustering y clasificación información relevante de jugadores de futbol.

Para comenzar cargue el dataset señalado y visualice a traves de un head los atributos que posee el dataset.

In [76]:
# df_players = pd.read_csv('./stats_players.csv').head(20)
# df_players

De la tabla se reconoce:

1. `Name`: Nombre del jugador
2. `Nationality`: Nacionalidad del jugador
3. `National_Position`: Posición en la selección, este valor solo existe cuando el jugador es seleccionado nacional.
4. `Club_Position`: Posición del jugador. Este parámetro tiene valor en gran parte de los jugadores, existen casos como `Res` o `Sub` que no los deberá considerar en la respectiva sección.
5. `Height`: Altura del jugador en centímetros. Si nota esta variable tiene un string, transforme a int.
6. `Weight`: Peso del jugador en kilogramos. Si nota esta variable tiene un string, transforme a int.
7. `Preffered_Foot`: Pie con que prefiere patear.
8. `Age`: Edad
9. `Work_Rate`: Frecuencia con que trabaja.

Mientras que el resto de las características representan estadísticas personales (de acuerdo a la habilidad) para cada uno de los jugadores.


## 1.1 Predicción de seleccionados nacionales. (2 puntos)

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


### 1.1.1 Generación de Labels para la Clasificación

Tras el trágico despido de la mítica mascota de la población de Maipú, *Renacín*, decide adentrarse en la predicción futbolera. Para esto, utilizando los datos cargados en las celdas anteriores, le consulta si es posible predecir **si un jugador de football será seleccionado nacional en base a sus estadísticas**

En este primer experimento de clasificación comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. Hecho esto, ¿Como se distribuyen los datos para cada clase?, comente lo que observa.


In [62]:
# DF de sólo atributos
features = df_players.drop(columns=['National_Position'])

# Columna target
labels = df_players['National_Position']

# Observaciones nulas (no seleccionados)
lmask = labels.isnull()

# Transformamos la columna target a one hot
labels = labels.fillna(0) # 0 para no seleccinados
labels[~lmask]=1          # 1 pára seleccionados

# Actualizamos la información en la base de datos original
df_players.National_Position = labels 
df_players.head()

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,1,LW,185 cm,80 kg,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,1,RW,170 cm,72 kg,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,1,LW,174 cm,68 kg,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,1,ST,182 cm,85 kg,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,1,GK,193 cm,92 kg,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11


In [44]:
print(f"Cantidad de datos etiquetados como seleccionados (1):\t {(df_players.National_Position == 1).sum()}")
print(f"Cantidad de datos etiquetados como no-seleccionados (0): {(df_players.National_Position == 0).sum()}")

Cantidad de datos etiquetados como seleccionados (1):	 17
Cantidad de datos etiquetados como no-seleccionados (0): 3


<b>¿Como se distribuyen los jugadores? </b>

Respuesta: Son 17 jugadores para la clase seleccionados y 3 para la clase de no seleccionados, por lo que tenemos un desbalance bastante grande en las clases (15% versus 85%). Con tan pocos datos y tan alto desbalance, la tarea de clasificar podría no ser muy eficiente o entregar muy buenos resultados.

### 1.1.2 Camino a la Clasificación

<p align="center">
  <img src="http://vignette3.wikia.nocookie.net/inciclopedia/images/c/cd/Patada_Medel_Messi.gif/revision/latest?cb=20150705215005" width="300">
</p>



Si bien los atributos de interés se encuentran en su mayoría limpios, existen valores mal ingresados al `DataFrame`. Para esto: 

1. Transforme a enteros todos los valores numéricos de la tabla que considere pertinentes. Por ejemplo: El peso aparece como strings y además estos continene la unidad. En este caso, debería transformar estos valores a enteros. Puede usar directamente funciones sobre el DataFrame (como `apply`) o el submódulo de las Series `str`.
2. Luego, aplique un ```ColumnTransformer``` para pre-procesar los datos del DataFrame de la siguiente forma:

Aplicar ```OneHotEncoder()``` sobre los atributos:

```python
['Preffered_Foot', 'Work_Rate']
```

Aplicar ```MinMaxScaler()``` sobre los atributos:

```python
['Age']
```

Aplicar ```StandardScaler()``` sobre los atributos:

```python
['Height', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Reactions','Interceptions', 'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys']
```

**Nota 2**: Puede copiar/pegar directamente las columnas en el `ColumnTransformer`.


Hecho lo anterior, defina al menos 3 pipelines para la clasificación donde anexe las transformaciones señaladas variando entre cada pipeline solo los clasificadores (puede escoger cualquiera de los clasificadores vistos en clases).

**To-Do:**

- [X] Modificar valores mal ingresados.
- [ ] Aplicar ```ColumnTransformer``` sobre los datos a través de un pipeline.
- [ ] Crear 3 pipelines con diferentes clasificadores.
- [ ] No entrenar los datos aún.

**Nota:** No es necesario entrenar los clasificadores aún.

In [78]:
# 1. Transforme a enteros todos los valores numéricos de la tabla que considere pertinentes
to_int = lambda s: int(s.split()[0] if type(s)==str else s) # func aux
df_players['Height']=df_players['Height'].apply(to_int)
df_players['Weight']=df_players['Weight'].apply(to_int)

# 2. Aplique un ColumnTransformer para pre-procesar los datos del DataFrame de la siguiente forma:

# Features
onehot_features = ['Preffered_Foot', 'Work_Rate']
minmax_features = ['Age']
stand_features  = ['Height', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Reactions','Interceptions', 'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys']

# Transformer
preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(), onehot_features),
        ('MinMax', MinMaxScaler(), minmax_features), 
        ('StandardScaler', StandardScaler(), stand_features)
        ])

# 3. Defina al menos 3 pipelines para la clasificación donde anexe las transformaciones señaladas variando entre cada pipeline solo los clasificadores (puede escoger cualquiera de los clasificadores vistos en clases).

pipe1 = Pipeline([('preprocessing',preprocessing_transformer),('Arbol', DecisionTreeClassifier(criterion="entropy"))])

pipe2 = Pipeline([('preprocessing',preprocessing_transformer),('K-vecinos', KNeighborsClassifier(n_neighbors=3))])##Buscar otro no funca

pipe2

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('OneHotEncoder',
                                                  OneHotEncoder(),
                                                  ['Preffered_Foot',
                                                   'Work_Rate']),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['Age']),
                                                 ('StandardScaler',
                                                  StandardScaler(),
                                                  ['Height', 'Weak_foot',
                                                   'Skill_Moves',
                                                   'Ball_Control', 'Dribbling',
                                                   'Marking', 'Sliding_Tackle',
                                                   'Standing_Tackle',
                                                   'Agg

In [60]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

to_int = lambda s: int(s.split()[0])
# def To_int(a):
#     b=a.split()
#     b=int(b[0])
#     return b
# a1=['Preffered_Foot', 'Work_Rate']
# a2=['Age']
# a3=['Height', 'Weak_foot', 'Skill_Moves', 'Ball_Control', 'Dribbling',
#                                               'Marking', 'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Reactions','Interceptions', 'Vision', 'Composure', 'Crossing', 'Short_Pass', 'Long_Pass',
#                                               'Acceleration', 'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing',

#                                               'Long_Shots', 'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys']
# df_player['Height']=df_player['Height'].apply(to_int)
# df_player['Weight']=df_player['Weight'].apply(to_int)
le=preprocessing.LabelEncoder()
features.Preffered_Foot=le.fit_transform(features.Preffered_Foot)##1:derecho 0:izquierdo, repetir lo mismo si encuentran otra 
features.Work_Rate=le.fit_transform(features.Work_Rate)##Es importante pero no entiendo que significa
# display(features.head(20))
# ##Columns_transformer:
# preprocessing_transformer = ColumnTransformer(
#     transformers=[
#         ('OneHotEncoder', OneHotEncoder(),  a1),
#         ('MinMax', MinMaxScaler(),a2), 
#         ('StandardScaler', StandardScaler(), a3)])

# pipaso1=Pipeline([('preprocessing',preprocessing_transformer),('Arbol',DecisionTreeClassifier(criterion="entropy"))])

# pipaso2=Pipeline([('preprocessing',preprocessing_transformer),('K-vecinos',KNeighborsClassifier(n_neighbors=3))])##Buscar otro no funca

# pipaso3=Pipeline([('preprocessing',preprocessing_transformer),('Vectores',svm.SVC())])###ver funcion de desicion/que significa
# print(features.shape)
df_players

AttributeError: 'int' object has no attribute 'split'

In [39]:
print(features.shape)

(17588, 38)


### 1.1.3 Métricas de evaluación + Opcional

<p align="center">
  <img src="https://media4.giphy.com/media/3orif8r5vpbAdQU3iU/200w.gif?cid=82a1493btc1h43n1h4c59p13kjv6adugso4lo4292zbgt66b&rid=200w.gif&ct=g" width="300">
</p>

Interesado por saber que tan bien funciona su clasificador, Renacin le solicita que construya **a mano** las siguientes métricas de evaluación para el caso de **clasificación binaria**:

- Accuracy (Exactitud):

$$Accuracy = \dfrac{número \, de \, predicciones \, correctas}{número \, de \,predicciones \, totales}$$


- Precision: 

$$Precision = \dfrac{Verdaderos \, positivos}{Verdaderos \, positivos + Falsos \, positivos}$$

- Recall:

$$Recall = \dfrac{Verdaderos \, positivos}{Verdaderos \, positivos + Falsos \,negativos}$$

<p align="center">
  <img src="https://www.researchgate.net/publication/336402347/figure/fig3/AS:812472659349505@1570719985505/Calculation-of-Precision-Recall-and-Accuracy-in-the-confusion-matrix.ppm" width="350">
</p>

**To-Do:**

- [ ] Crear funciones para las métricas para clasificación binaria: Accuracy, Precision y Recall. 
- [ ] Compruebe el correcto funcionamiento de sus métricas a través de los test.

**Hint:** Utilice confusion_matrix para obtener TP, FP, FN y TN.



In [8]:
from sklearn.metrics import confusion_matrix

def accuracy(y_true, y_pred):
        cm=confusion_matrix(y_true, y_pred,labels=[1,0])
        ac=(cm[0][0]+cm[1][1])/(cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0])
        return ac
        

def presicion(y_true, y_pred):
    cm=confusion_matrix(y_true, y_pred,labels=[1,0])
    p=cm[0][0]/(cm[0][0]+cm[0][1])
    return p
  

def recall(y_true, y_pred):
    cm=confusion_matrix(y_true, y_pred,labels=[1,0])
    rc=cm[0][0]/(cm[0][0]+cm[1][0])
    return rc
 

In [9]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
y_true = [0, 1, 0, 0, 1, 0]
y_pred = [0, 0, 1, 0, 0, 1]
# Tests de metricas
assert accuracy_score(y_true, y_pred) == accuracy(y_true, y_pred)
assert recall_score(y_true, y_pred) == recall(y_true, y_pred)
assert precision_score(y_true, y_pred) == presicion(y_true, y_pred)

### 1.1.4 Entrenemos los pipelines

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="300">
</p>


Entrene en base al pipeline generado en los pasos anteriores, procure separar los datos de entrenamiento en un conjunto de entrenamiento y de prueba (la proporción queda a su juicio). Una vez entrenado, muestre los resultados para cada uno de los clasificadores y comente lo que observa de los resultados obtenidos usando las métricas creadas anteriormente, ¿podrían mejorarse los resultados?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar el pipeline.
- [ ] Utilizar las métricas codificadas anteriormente y analizar los resultados.


In [11]:
from sklearn.model_selection import train_test_split
bongazo=[pipaso1,pipaso2,pipaso3]
for i in range(len(bongazo)):
    features_final=bongazo[i][0].fit_transform(features).copy()
    features_final= pd.DataFrame(features_final)
    ##Separar data
    X_train, X_test, y_train, y_test = train_test_split(
        features_final, labels, test_size=0.33, shuffle=True, stratify=labels )
    y_train=y_train.astype('int')##No puede ser un objeto
    y_test=y_test.astype('int')
    ##Entrenar
    Tr=bongazo[i][1].fit(X_train, y_train)
    y_pred1 = Tr.predict(X_test)
    print(f"Accuracy:{accuracy(y_test, y_pred1)}")##Poner nombres apropiados para que se vea lindo
    print(f"Recall:{recall(y_test, y_pred1)}")
    print(f"Presicion:{presicion(y_test, y_pred1)}")

Accuracy:0.9000861326442722
Recall:0.206266318537859
Presicion:0.22253521126760564
Accuracy:0.9331610680447889
Recall:0.35135135135135137
Presicion:0.10985915492957747
Accuracy:0.9388458225667528
Recall:nan
Presicion:0.0


<ipython-input-8-fa2bd39f2807>:17: RuntimeWarning: invalid value encountered in longlong_scalars
  rc=cm[0][0]/(cm[0][0]+cm[1][0])


**Respuesta**:
Joder

```
Comente de resultados observados y si es posible mejorar los resultados.
```

## 1.2 Predicción de posiciones de jugadores. (2 puntos)

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="300">
</p>

### 1.2.1 Creación de nuevas labels.



No conforme con el trabajo de la sección 1.1, Renacin les *ordena* que construyan un clasificador nuevo, pero para predecir la posición de los jugadores del dataset. Para lograr esto, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, con esto les recalca que no deben utilizar los valores ```Sub``` y ```Res``` de este atributo, pidiendo que agrupen los valores en los siguientes grupos:

```python
Ataque=['ST', 'CAM','RS','LF','LAM','LS','RF','RAM','CF']
MedioCampistas=['RCM','LW', 'RW', 'LCM','LM', 'RM', 'CM']
Defensas=['LCB','RB','RCB', 'LDM','LB','CDM','RDM','CB','RWB','LWB']
Arquero =['GK']
```

Con lo señalado, Renacin les señala que no deberian utilizar el label utilizado en la sección 1.1, ya que el problema de clasificación ahora es de posiciones.

**To-Do**:

- [ ] Aplique las etiquetas de ```Ataque```, ```MedioCampista```, ```Defensa``` y ```Arquero``` en los valores señalados en esta sección.
- [ ] Muestre como se distribuyen los datos etiquetados.

In [12]:
#display(features.head(20))
Ataque=['ST', 'CAM','RS','LF','LAM','LS','RF','RAM','CF']
MedioCampistas=['RCM','LW', 'RW', 'LCM','LM', 'RM', 'CM']
Defensas=['LCB','RB','RCB', 'LDM','LB','CDM','RDM','CB','RWB','LWB']
Arquero =['GK']
con=[Ataque,MedioCampistas,Defensas,Arquero]
for i in range(4):
    for val in con[i]: 
        features.loc[features.Club_Position==val,'Club_Position']=i
     
features.head(20)


,Name,Nationality,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,1,185,80,1,32,1,4,5,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,1,170,72,0,29,8,4,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,1,174,68,1,25,2,5,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,0,182,85,1,30,2,4,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,3,193,92,1,31,8,4,1,...,52,78,25,25,13,16,14,11,47,11
5,De Gea,Spain,3,193,82,1,26,8,3,1,...,57,67,21,31,13,12,21,19,40,13
6,Robert Lewandowski,Poland,0,185,79,1,28,2,4,3,...,78,84,85,86,91,82,77,76,81,86
7,Gareth Bale,Wales,1,183,74,0,27,2,3,4,...,77,85,86,91,87,90,86,85,76,76
8,Zlatan Ibrahimović,Sweden,0,195,95,1,35,7,4,4,...,86,72,80,93,90,88,82,82,91,93
9,Thibaut Courtois,Belgium,3,199,91,0,24,8,3,1,...,61,68,13,36,14,17,19,11,27,12


In [47]:
###Hay que quitar los subs y el otro:
df22=(features["Club_Position"]==0) | (features["Club_Position"]==1) | (features["Club_Position"]==2) | (features["Club_Position"]==3)



,Name,Nationality,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
10,Jérôme Boateng,Germany,Sub,192.0,90.0,1.0,28.0,8.0,4.0,2.0,...,58.0,75.0,86.0,79.0,34.0,58.0,56.0,31.0,46.0,53.0
58,Claudio Marchisio,Italy,Sub,180.0,75.0,1.0,31.0,0.0,4.0,4.0,...,81.0,67.0,68.0,82.0,65.0,82.0,78.0,78.0,78.0,76.0
61,Cesc Fàbregas,Spain,Sub,175.0,74.0,1.0,29.0,2.0,3.0,3.0,...,65.0,68.0,74.0,77.0,76.0,75.0,77.0,81.0,80.0,81.0
62,Franck Ribéry,France,Sub,170.0,72.0,1.0,33.0,2.0,4.0,5.0,...,88.0,50.0,41.0,76.0,76.0,73.0,83.0,81.0,78.0,80.0
63,Santi Cazorla,Spain,Sub,168.0,66.0,1.0,32.0,8.0,5.0,4.0,...,85.0,70.0,58.0,78.0,77.0,82.0,86.0,78.0,80.0,72.0


In [15]:
print(f"Cantidad de datos etiquetados como delantero: {(features.Club_Position == 0).sum()}")
print(f"Cantidad de datos etiquetados como defensa: {(features.Club_Position == 1).sum()}")
print(f"Cantidad de datos etiquetados como medio campistas: {(features.Club_Position== 2).sum()}")
print(f"Cantidad de datos etiquetados como arqueros: {(features.Club_Position == 3).sum()}")

Cantidad de datos etiquetados como delantero: 1252
Cantidad de datos etiquetados como defensa: 1880
Cantidad de datos etiquetados como medio campistas: 3185
Cantidad de datos etiquetados como arqueros: 632


### 1.2.2 Entrenar un pipeline

Aplicando el mismo preprocesamiento solicitado en 1.1.2, genere un pipeline con un clasificador a gusto y evalúe al modelo de clasificación utilizando la **librería `scikit-learn`** (esto debido a que este es un problema multi clase y las funciones solicitadas anteriormente era para una clasificación binaria).

De los resultados obtenidos, ¿El modelo pudo clasificar mejor este problema o el de la sección 1.1?, Comente sus resultados.

**To-Do**:

- [ ] Genere un pipeline similar al visto en la sección 1.1.2.
- [ ] Aplique las métricas ofrecidas por la librería scikit-learn.
- [ ] Responda la pregunta del enunciado.

In [13]:
features1 = features.drop(columns=['Club_Position'])
labels1 = features.loc[:, 'Club_Position']
labels[~lmask]=1
features1=pd.concat([features1,labels],axis=1)

preprocessing_transformer = ColumnTransformer(  ###Lo mismo
    transformers=[
        ('OneHotEncoder', OneHotEncoder(),  a1),
        ('MinMax', MinMaxScaler(),a2), 
        ('StandardScaler', StandardScaler(), a3)])


pipasofinal=Pipeline([('preprocessing',preprocessing_transformer),('Arbol',DecisionTreeClassifier(criterion="entropy"))])

features1=pipasofinal[0].fit_transform(features1).copy()
features1= pd.DataFrame(features1)

X1_train, X1_test, y1_train, y1_test = train_test_split(
        features1, labels1, test_size=0.33, shuffle=True, stratify=labels )
    y1_train=y1_train.astype('int')##No puede ser un objeto
    y1_test=y1_test.astype('int')
    Tr1=pipasofinal[1].fit(X1_train, y1_train)
    y_pred2 = Tr1.predict(X1_test)
    print(f"Accuracy:{accuracy_score(y1_test, y_pred2)}")##Poner nombres apropiados para que se vea lindo
    print(f"Recall:{recall_score(y1_test, y_pred2)}")
    print(f"Presicion:{presicion_score(y1_test, y_pred2)}")


**Respuesta**:

```
Comente de resultados observados.
```

## 1.3 Clustering Sobre el Dataset (2 puntos)

<p align="center">
  <img src="https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F1832816%2F349289c622faaff3afc517a46fcf4c71%2Fk-means.jpg?generation=1577258176400530&alt=media" width="200">
</p>

### 1.3.1 Elección de numero de clusters

<p align="center">
  <img src="https://media1.tenor.com/images/a85e9df2f5784d6513dce853ba9d5755/tenor.gif?itemid=18229271" width="200">
</p>





Entrene un k-means con 3, 4 y 5 clusters. ¿Las agrupaciones son lógicas usando un análisis visual?, Grafique los clusters con el código dispuesto más abajo y comente cual de las opciones presenta una mejor agrupación.

En base al análisis realizado, ubique la posición de los cluster en la cancha. Solo señale a que posición equivaldría tomando de referencia:

- Ataque
- Medio campistas
- Defensas
- Arquero 

**To-Do**:

- [ ] Escoger un numero de cluster y justificar elección.
- [ ] Comentar si las agrupaciones son lógicas a través de un análisis visual.
- [ ] Graficar los resultados obtenidos y visualizar las agrupaciones obtenidas.


In [ ]:
from umap import UMAP

def graficar_clusters(clusters, preprocessed_data = preprocessed_data,posicion = df_players.Club_Position.values):
  # Entreguele a esta función solo los clusters, el resto de parametros debería funcionar por default.
  umap = UMAP()
  projections = umap.fit_transform(preprocessed_data)
  fig = px.scatter(x=projections[:,0], y=projections[:,1], color=clusters, hover_data={'posicion': posicion}) 
  fig.show()

In [ ]:
### Código aquí ###

**Respuesta**:

```
Comente de resultados observados.
```

### 1.3.2 Búsqueda Número Óptimo de Clusters: Método del Codo **(Opcional 0.5 puntos)**

<p align="center">
  <img src="https://c.tenor.com/fk4p5HzTi5gAAAAC/elbow-high-five.gif" width="300">
</p>





Renancin muy indeciso de sus resultados le comenta que desea realizar un último trabajo. En esta última tarea le solicita la agrupación de los datos obtenidos en la sección 1.2 utilizando K-means. Para esto usted deberá visualizar a través del método del codo el número óptimo de agrupaciones para los datos. 

**Hint:** Revisar las clases y encontrar el código.

**To-Do**:

- [ ] Pre-procesar los datos con el mismo preprocesamiento aplicado en 1.2
- [ ] Utilizar el método del codo para obtener el número de clusters.

In [ ]:
### Código aquí ###

**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>